In [13]:
import pandas as pd 
import plotly.express as px
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';',parse_dates=['timestamp'])

In [17]:
df=dfo.copy()
c=list(df.columns)
c.append("Dev")
df.columns=c
df=df[df['Dev']!=0]
df['Dev']=df['Dev']!=1
df['Dev']=df['Dev'].astype('int')
df_clean=df.copy()

ValueError: Length mismatch: Expected axis has 42 elements, new values have 43 elements

In [16]:
df_clean

,Dev,LevelMainTank ValueY,YC10001_MV ValueY,PI12002_PV_Out ValueY,PI12003_PV_Out ValueY,YC14001_MV ValueY,FIC14002_MV ValueY,FIC14002_PV_Out ValueY,FIC14002_SP ValueY,YC14006_MV ValueY,...,LIC22002_PV_Out ValueY,LIC22002_SP ValueY,LIC21002_MV ValueY,LIC21002_PV_Out ValueY,LIC21002_SP ValueY,PIC14007_MV ValueY,PIC14007_PV_Out ValueY,PIC14007_SP ValueY,YC14008_Ctrl ValueY,PI10003_PV_Out ValueY
0,0,149.985779,100,0.070602,0.087963,100.0,100.0,0.0,2000.0,12.5,...,1.966218,2,0.611938,1.979745,15,10.0,0.127008,0.2,0,0.001710
1,0,149.985779,100,0.070602,0.086806,100.0,100.0,0.0,2000.0,12.5,...,1.967810,2,1.151489,1.979745,15,10.0,0.127021,0.2,0,0.001715
2,0,150.027740,100,0.071759,0.103588,100.0,100.0,0.0,2000.0,12.5,...,1.958261,2,11.073669,1.986111,15,10.0,0.127021,0.2,0,0.001712
3,0,150.043427,100,0.072338,0.173611,100.0,100.0,0.0,2000.0,12.5,...,1.958261,2,11.819598,1.982133,15,10.0,0.126945,0.2,0,0.001698
4,0,150.015564,100,0.065972,0.326968,100.0,100.0,0.0,2000.0,12.5,...,1.967810,2,22.069590,1.982133,15,10.0,0.126945,0.2,0,0.001700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278487,1,112.293930,100,-0.899306,-0.582176,100.0,100.0,0.0,2000.0,12.5,...,6.972077,2,800.000000,7.470197,15,10.0,-0.009348,0.2,0,-0.009062
278488,1,112.352432,100,-0.902199,-0.586227,100.0,100.0,0.0,2000.0,12.5,...,6.968894,2,800.000000,7.478154,15,10.0,-0.009345,0.2,0,-0.009085
278489,1,112.400436,100,-0.901042,-0.565394,100.0,100.0,0.0,2000.0,12.5,...,6.961733,2,800.000000,7.471788,15,10.0,-0.009364,0.2,0,-0.009090
278490,1,112.441315,100,-0.902778,-0.578704,100.0,100.0,0.0,2000.0,12.5,...,6.957755,2,800.000000,7.475767,15,10.0,-0.009384,0.2,0,-0.009099


In [15]:
dfo.groupby('CuStepNo ValueY').aggregate({'timestamp':'min','timestamp':'max'})

,timestamp
CuStepNo ValueY,
1,2018-10-31 14:34:36
2,2018-10-31 13:52:25
3,2018-10-31 14:29:30
7,2018-10-31 14:35:49
8,2018-10-31 15:00:15
